In [1]:
!pip install nbformat
!pip install scikit-learn


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.arima.model import ARIMA
from scipy.ndimage import gaussian_filter1d
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import pickle
import warnings
import json
warnings.filterwarnings('ignore')

# Configure Prophet to use cmdstanpy backend
import logging
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

# --- CONFIGURATION ---
TRAIN_YEARS = 10  # Use 10 years data to train
FORECAST_DAYS = 91       # USER INPUT
CV_HORIZON = '91 days'
CV_PERIOD = '182 days'
CV_INITIAL = '3285 days'

# SMOOTHING CONFIGURATION
SMOOTHING_STRENGTH = 0.7
USE_HYBRID = True # Use Prophet + ARIMA hybrid approach
USE_INTERACTIVE_CHARTS = False  # Disable charts to speed up training

# MODEL SAVING CONFIGURATION
SAVE_MODELS = True
MODEL_FOLDER = 'SaveModels'  # Folder to save models
USE_BEST_PARAMS = True  # Use best parameters from optimize_prophet.py

# Stock files list
df_files = [
    'dataset/AMZN_stock_data.csv', 'dataset/AAPL_stock_data.csv', 'dataset/MMM_stock_data.csv', 'dataset/BA_stock_data.csv',
    'dataset/AXP_stock_data.csv', 'dataset/CAT_stock_data.csv', 'dataset/CRM_stock_data.csv', 'dataset/CSCO_stock_data.csv',
    'dataset/CVX_stock_data.csv', 'dataset/DIS_stock_data.csv', 'dataset/GS_stock_data.csv', 'dataset/HD_stock_data.csv',
    'dataset/HON_stock_data.csv', 'dataset/IBM_stock_data.csv', 'dataset/JNJ_stock_data.csv', 'dataset/JPM_stock_data.csv',
    'dataset/MCD_stock_data.csv', 'dataset/MSFT_stock_data.csv', 'dataset/NVDA_stock_data.csv', 'dataset/PG_stock_data.csv',
    'dataset/SHW_stock_data.csv', 'dataset/TRV_stock_data.csv', 'dataset/UNH_stock_data.csv', 'dataset/V_stock_data.csv'
]

def add_technical_features(df):
    # Exponential moving averages
    df['EMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EMA_21'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    
    # Smooth volatility
    df['Volatility_21'] = df['Close'].rolling(window=21, min_periods=1).std()
    df['Volatility_21'] = df['Volatility_21'].ewm(span=5, adjust=False).mean()
    
    # Smooth momentum
    df['Momentum_14'] = df['Close'].pct_change(periods=14).fillna(0)
    df['Momentum_14'] = df['Momentum_14'].ewm(span=5, adjust=False).mean()
    
    # Volume indicators
    df['Volume_EMA_21'] = df['Volume'].ewm(span=21, adjust=False).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Volume_EMA_21']
    df['Volume_Ratio'] = df['Volume_Ratio'].ewm(span=5, adjust=False).mean()
    
    # Fill any remaining NaN values
    df = df.fillna(method='bfill').fillna(method='ffill')
    
    return df

def smooth_forecast(forecast_series, strength=0.7):
    sigma = strength * 5
    smoothed = gaussian_filter1d(forecast_series, sigma=sigma)
    return smoothed

def fit_arima_trend(data, order=(2, 1, 2)):
    try:
        model = ARIMA(data, order=order, enforce_stationarity=False, enforce_invertibility=False)
        fitted_model = model.fit()
        return fitted_model
    except:
        model = ARIMA(data, order=(1, 1, 1), enforce_stationarity=False)
        return model.fit()

def save_forecast_results(forecast_df, stock_name, best_params=None, metrics=None, folder='SaveModels'):
    """Save forecast data, parameters, and metrics to files"""
    try:
        if not os.path.exists(folder):
            os.makedirs(folder)
        
        # Convert datetime to strings for pickling compatibility
        forecast_copy = forecast_df.copy()
        forecast_copy['ds'] = forecast_copy['ds'].astype(str)
        
        # Save forecast data
        forecast_filename = f"{stock_name}_forecast.pkl"
        forecast_filepath = os.path.join(folder, forecast_filename)
        with open(forecast_filepath, 'wb') as f:
            pickle.dump(forecast_copy, f)
        print(f" ✓ Forecast saved: {forecast_filename}")
        
        # Save best parameters
        if best_params:
            params_filename = f"{stock_name}_best_params.pkl"
            params_filepath = os.path.join(folder, params_filename)
            with open(params_filepath, 'wb') as f:
                pickle.dump(best_params, f)
            print(f" ✓ Parameters saved: {params_filename}")
        
        # Save metrics
        if metrics:
            metrics_filename = f"{stock_name}_metrics.pkl"
            metrics_filepath = os.path.join(folder, metrics_filename)
            with open(metrics_filepath, 'wb') as f:
                pickle.dump(metrics, f)
            print(f" ✓ Metrics saved: {metrics_filename}")
        
        return True
        
    except Exception as e:
        print(f" ✗ Error saving forecast for {stock_name}: {e}")
        import traceback
        traceback.print_exc()
        return False

def load_best_params(stock_name, folder='SaveModels'):
    """Load best parameters from file"""
    try:
        params_filename = f"{stock_name}_best_params.pkl"
        params_filepath = os.path.join(folder, params_filename)
        
        if not os.path.exists(params_filepath):
            return None
        
        with open(params_filepath, 'rb') as f:
            params = pickle.load(f)
        
        return params
    except Exception as e:
        print(f" Warning: Could not load parameters for {stock_name}: {e}")
        return None

def optimize_prophet_model(train_df, stock_name, best_params=None):
    """Create and configure Prophet model with best parameters or defaults"""
    try:
        if best_params:
            print(f" ✓ Using best-tuned parameters for {stock_name}")
            model = Prophet(
                changepoint_prior_scale=float(best_params['changepoint_prior_scale']),
                seasonality_prior_scale=float(best_params['seasonality_prior_scale']),
                seasonality_mode=str(best_params['seasonality_mode']),
                changepoint_range=float(best_params.get('changepoint_range', 0.8)),
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=True,
                interval_width=0.95
            )
        else:
            volatility = train_df['y'].pct_change().std()
            
            if volatility > 0.03:
                model = Prophet(
                    changepoint_prior_scale=0.03,
                    seasonality_prior_scale=3,
                    seasonality_mode='multiplicative',
                    changepoint_range=0.85,
                    daily_seasonality=False,
                    weekly_seasonality=False,
                    yearly_seasonality=True,
                    interval_width=0.95
                )
            elif volatility > 0.015:
                model = Prophet(
                    changepoint_prior_scale=0.02,
                    seasonality_prior_scale=5,
                    seasonality_mode='additive',
                    changepoint_range=0.82,
                    daily_seasonality=False,
                    weekly_seasonality=False,
                    yearly_seasonality=True,
                    interval_width=0.95
                )
            else:
                model = Prophet(
                    changepoint_prior_scale=0.01,
                    seasonality_prior_scale=8,
                    seasonality_mode='additive',
                    changepoint_range=0.80,
                    daily_seasonality=False,
                    weekly_seasonality=False,
                    yearly_seasonality=True,
                    interval_width=0.95
                )
        
        model.add_country_holidays(country_name='US')
        model.add_seasonality(name='monthly', period=30.5, fourier_order=3)
        model.add_seasonality(name='quarterly', period=91.25, fourier_order=4)
        
        regressor_cols = ['EMA_7', 'EMA_21', 'EMA_50', 'Volatility_21', 'Momentum_14', 'Volume_Ratio']
        
        for col in regressor_cols:
            if col in train_df.columns:
                model.add_regressor(col, standardize='auto')
        
        return model
    except Exception as e:
        print(f" ✗ Error creating Prophet model: {e}")
        raise

# ============================================================================
# MAIN TRAINING LOOP
# ============================================================================

metrics_summary = []
processed_count = 0

for stock_file in df_files:
    print("\n" + "=" * 80)
    print(f" 📊 Processing: {stock_file}")
    print("=" * 80)

    try:
        # --- Load data ---
        df = pd.read_csv(stock_file, sep='|')
        date_col = 'Date'
        
        df[date_col] = pd.to_datetime(df[date_col])
        df = df.sort_values(by=date_col).reset_index(drop=True)
        
        # --- Keep only last 10 years ---
        last_date = df[date_col].max()
        cutoff_date = last_date - pd.DateOffset(years=TRAIN_YEARS)
        df = df[df[date_col] >= cutoff_date].reset_index(drop=True)
        
        print(f" Data range: {df[date_col].min().date()} to {df[date_col].max().date()}")
        print(f" Total trading days: {len(df)}")
        
        # --- Add technical features ---
        df = add_technical_features(df)
        
        stock_name = os.path.basename(stock_file).split('_')[0]
        
        # Create training dataframe
        train = pd.DataFrame({'ds': df[date_col], 'y': df['Close']})
        
        regressor_cols = ['EMA_7', 'EMA_21', 'EMA_50', 'Volatility_21', 'Momentum_14', 'Volume_Ratio']
        
        for col in regressor_cols:
            if col in df.columns:
                train[col] = df[col]
        
        # --- Load best parameters ---
        best_params = None
        if USE_BEST_PARAMS:
            best_params = load_best_params(stock_name, MODEL_FOLDER)
        
        # --- Train Prophet model ---
        print(f" Creating Prophet model for {stock_name}...")
        model = optimize_prophet_model(train, stock_name, best_params)
        
        print(f" Fitting model...")
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            model.fit(train)
        
        # --- Create future dataframe ---
        future = model.make_future_dataframe(periods=FORECAST_DAYS)
        
        # Extrapolate regressors
        for col in regressor_cols:
            if col in train.columns:
                last_values = train[col].tail(30).values
                trend = np.mean(np.diff(last_values))
                last_value = train[col].iloc[-1]
                future_values = [last_value + trend * i * 0.5 for i in range(1, FORECAST_DAYS + 1)]
                future.loc[future['ds'] > train['ds'].max(), col] = future_values
                future.loc[future['ds'] <= train['ds'].max(), col] = train[col].values
        
        print(f" Generating forecast...")
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            forecast = model.predict(future)
        
        # --- Apply ARIMA hybrid ---
        if USE_HYBRID:
            try:
                recent_data = train['y'].tail(365).values
                arima_model = fit_arima_trend(recent_data, order=(1, 1, 1))
                arima_forecast = arima_model.forecast(steps=FORECAST_DAYS)
                future_prophet = forecast[forecast['ds'] > train['ds'].max()]['yhat'].values
                blended_forecast = 0.7 * future_prophet + 0.3 * arima_forecast
                forecast.loc[forecast['ds'] > train['ds'].max(), 'yhat'] = blended_forecast
                print(" ✓ Hybrid Prophet + ARIMA applied")
            except Exception as e:
                print(f" ⚠ ARIMA skipped: {e}")
        
        # --- Apply smoothing ---
        future_mask = forecast['ds'] > train['ds'].max()
        forecast.loc[future_mask, 'yhat'] = smooth_forecast(forecast.loc[future_mask, 'yhat'].values, strength=SMOOTHING_STRENGTH)
        forecast.loc[future_mask, 'yhat_lower'] = smooth_forecast(forecast.loc[future_mask, 'yhat_lower'].values, strength=SMOOTHING_STRENGTH * 0.8)
        forecast.loc[future_mask, 'yhat_upper'] = smooth_forecast(forecast.loc[future_mask, 'yhat_upper'].values, strength=SMOOTHING_STRENGTH * 0.8)
        
        # --- Evaluate model ---
        accuracy = 0.0
        mape = 0.0
        mae = 0.0
        rmse = 0.0
        
        try:
            total_days = (train['ds'].max() - train['ds'].min()).days
            
            if total_days >= 3650:
                print(f" Running cross-validation...")
                with warnings.catch_warnings():
                    warnings.simplefilter('ignore')
                    cv_df = cross_validation(
                        model,
                        initial='2920 days',
                        period='182 days',
                        horizon='91 days',
                        parallel="processes",
                        disable_tqdm=True
                    )
                
                mape = mean_absolute_percentage_error(cv_df['y'], cv_df['yhat']) * 100
                mae = mean_absolute_error(cv_df['y'], cv_df['yhat'])
                rmse = np.sqrt(mean_squared_error(cv_df['y'], cv_df['yhat']))
                accuracy = 100 - mape
                
                print(f" ✓ Cross-validation Results:")
                print(f"   Accuracy: {accuracy:.2f}%")
                print(f"   MAPE: {mape:.2f}%")
        except Exception as e:
            print(f" ⚠ Cross-validation skipped: {str(e)[:100]}")
            accuracy = 0.0
        
        # Add metrics
        avg_price = df['Close'].mean() if len(df) > 0 else 0.0
        metrics_summary.append({
            'Stock': stock_name,
            'Accuracy (%)': accuracy,
            'MAPE (%)': mape,
            'MAE ($)': mae,
            'RMSE ($)': rmse,
            'Avg_Price ($)': avg_price
        })
        
        # --- Save forecast results ---
        if SAVE_MODELS:
            forecast_to_save = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
            metrics_dict = {'accuracy': accuracy, 'mape': mape, 'mae': mae, 'rmse': rmse}
            save_forecast_results(forecast_to_save, stock_name, best_params, metrics_dict, MODEL_FOLDER)
        
        processed_count += 1
        print(f" ✓ Successfully processed {stock_name}")
        
        # Delete model to free memory
        del model
        del forecast
        
    except Exception as e:
        print(f" ✗ Error processing {stock_file}: {str(e)[:150]}")
        try:
            stock_name = os.path.basename(stock_file).split('_')[0]
        except:
            stock_name = "UNKNOWN"
        metrics_summary.append({
            'Stock': stock_name,
            'Accuracy (%)': 0.0,
            'MAPE (%)': 0.0,
            'MAE ($)': 0.0,
            'RMSE ($)': 0.0,
            'Avg_Price ($)': 0.0
        })

# ============================================================================
# SUMMARY RESULTS
# ============================================================================

print("\n" + "=" * 100)
print(" 📊 TRAINING COMPLETE")
print("=" * 100)
print(f" Total stocks processed: {processed_count}/{len(df_files)}")

if len(metrics_summary) > 0:
    metrics_df = pd.DataFrame(metrics_summary)
    metrics_df = metrics_df.sort_values(by='Accuracy (%)', ascending=False).reset_index(drop=True)

    print("\n" + "=" * 100)
    print(" 📊 FINAL PERFORMANCE SUMMARY")
    print("=" * 100)
    print(metrics_df.to_string(index=False))

    high_accuracy_stocks = metrics_df[metrics_df['Accuracy (%)'] > 0.0]
    print(f"\n ⭐ STOCKS SUCCESSFULLY TRAINED: {len(high_accuracy_stocks)}/{len(metrics_df)}")
    
    if len(high_accuracy_stocks) > 0:
        print(high_accuracy_stocks[['Stock', 'Accuracy (%)', 'MAPE (%)']].to_string(index=False))

    avg_accuracy = metrics_df['Accuracy (%)'].mean()
    print(f"\n 📈 Average Accuracy: {avg_accuracy:.2f}%")
    print(f" ✓ All forecasts saved to {MODEL_FOLDER}/")
else:
    print("\n ✗ No metrics collected")



 📊 Processing: dataset/AMZN_stock_data.csv
 Data range: 2015-10-12 to 2025-10-10
 Total trading days: 2515
 Creating Prophet model for AMZN...
 ✗ Error creating Prophet model: 'Prophet' object has no attribute 'stan_backend'
 ✗ Error processing dataset/AMZN_stock_data.csv: 'Prophet' object has no attribute 'stan_backend'

 📊 Processing: dataset/AAPL_stock_data.csv
 Data range: 2015-10-12 to 2025-10-10
 Total trading days: 2515
 Creating Prophet model for AAPL...
 ✗ Error creating Prophet model: 'Prophet' object has no attribute 'stan_backend'
 ✗ Error processing dataset/AAPL_stock_data.csv: 'Prophet' object has no attribute 'stan_backend'

 📊 Processing: dataset/MMM_stock_data.csv
 Data range: 2015-10-12 to 2025-10-10
 Total trading days: 2515
 Creating Prophet model for MMM...
 ✗ Error creating Prophet model: 'Prophet' object has no attribute 'stan_backend'
 ✗ Error processing dataset/MMM_stock_data.csv: 'Prophet' object has no attribute 'stan_backend'

 📊 Processing: dataset/BA_stoc